In [1]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
from collections import Counter
import math
import copy


data = pd.read_csv('Binary_data.csv')
# data = pd.read_csv('/Users/macbookair/workspace/COM804/Approach_1/Multi_data.csv')

### Binary Pred

In [2]:
X = data.iloc[:,:-1]
y = data.iloc[:, -1]

In [3]:
X.shape

(969, 22)

In [27]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=31)
# X_val, X_test, y_val, y_test = train_test_split(X_, y_, test_size= 0.5, random_state=31)

In [28]:
X_train = X_train.reset_index(drop=True)
y_train = y_train.reset_index(drop=True)
# X_val = X_val.reset_index(drop=True)
# y_val = y_val.reset_index(drop=True)
X_test = X_test.reset_index(drop=True)
y_test = y_test.reset_index(drop=True)

In [29]:
print("Training size: ", X_train.shape[0])
print("Test size: ", X_test.shape[0])

Training size:  678
Test size:  291


In [30]:
def normalize_features(X, X_train):
    
    eps = 1e-12
    X_min = X_train.min(axis=0)
    X_max = X_train.max(axis=0)
    diff  = np.maximum(X_max - X_min, eps)
    X_norm = (X - X_min) / diff
    
    return X_norm

# X_train_norm = normalize_features(X_train, X_train)
# X_val_norm = normalize_features(X_val, X_train)

In [31]:
X_train_norm = normalize_features(X_train, X_train)
# X_val_norm = normalize_features(X_val, X_train)
X_test_norm = normalize_features(X_test, X_train)


In [32]:
X_train_norm.head()

,Sex,Age,Systolic_Pressure,BMI,CKD_Cause,Hemoglobin,Albumin,Creatinine,eGFR,CKD_Stage,CKD_Risk,Dipstick_Proteinuria,Proteinuria,Occult_Blood_in_Urine,Protein_Creatinine_Ratio,UPCR_Severity,Hypertension,Previous_CVD,Diabetes,RAAS_Inhibitor,Calcium_Channel_Blocker,Diuretics
0,0.0,0.000000,0.223684,0.237548,0.666667,0.731092,0.864865,0.091116,1.000000,0.000000,0.090909,0.4,1.0,0.0,0.009415,0.5,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.861111,0.355263,0.291188,0.333333,0.521008,0.837838,0.476082,0.077676,1.000000,1.000000,0.0,0.0,0.0,0.025273,1.0,1.0,0.0,0.0,1.0,0.0,1.0
2,0.0,0.611111,0.368421,0.241379,0.333333,0.915966,0.702703,0.148064,0.516477,0.333333,0.363636,0.2,0.0,0.0,0.018335,0.5,1.0,0.0,0.0,1.0,1.0,0.0
3,0.0,0.750000,0.250000,0.425287,0.333333,0.487395,0.648649,0.248292,0.317270,0.333333,0.363636,0.0,0.0,1.0,0.009911,0.5,1.0,0.0,0.0,1.0,0.0,0.0
4,0.0,0.708333,0.467105,0.517241,0.333333,0.537815,0.648649,0.179954,0.429138,0.333333,0.272727,0.0,0.0,1.0,0.000991,0.0,1.0,0.0,0.0,1.0,1.0,0.0


In [33]:
num_cols = ['Age', 'Systolic_Pressure', 'BMI', 'Hemoglobin', 'Albumin', 'Creatinine', 'eGFR', 'Protein_Creatinine_Ratio']
cat_cols = [c for c in X_train if c not in num_cols]

In [34]:
### printt the number of unique values in each feature
for i in X_train.columns:
    print(f"{i}: {X_train[i].nunique()}")

Sex: 2
Age: 66
Systolic_Pressure: 107
BMI: 169
CKD_Cause: 4
Hemoglobin: 107
Albumin: 36
Creatinine: 274
eGFR: 626
CKD_Stage: 4
CKD_Risk: 12
Dipstick_Proteinuria: 6
Proteinuria: 2
Occult_Blood_in_Urine: 2
Protein_Creatinine_Ratio: 315
UPCR_Severity: 3
Hypertension: 2
Previous_CVD: 2
Diabetes: 2
RAAS_Inhibitor: 2
Calcium_Channel_Blocker: 2
Diuretics: 2


In [35]:
## seperating the numerical columns and the categorical columns
cat_cols = [col for col in X_train.columns if X_train[col].nunique() <= 12]
num_cols = [c for c in X_train if c not in cat_cols]

In [36]:
# # Initialize the weights to 1
# W = {}
# for col in X_train.columns:
#     W[col] = 1.0
n = X_train_norm.shape[1]
W = np.ones(n,)

In [37]:
### similarity search
def local_sims(new_case, casebase, num_cols, cat_cols, eps=1e-12):
    sims = {}
    
    for col in num_cols:
            diff = np.abs(new_case[col] - casebase[col])
            f_min = casebase[col].min()
            f_max = casebase[col].max()
            f_range = np.maximum(f_max - f_min, eps)
            sims[col] = (1 - (diff / f_range)).clip(0, 1)
        
    for col in cat_cols:
            sims[col] = (casebase[col] == new_case[col]).astype(float)
    
    df_sim = pd.DataFrame(sims, index=casebase.index)
    ordered_cols = [c for c in casebase.columns if c in df_sim.columns]
    
    return df_sim[ordered_cols]

### gloabl similarity search

def global_sims(X, W, top_k):
    W = np.asarray(W, dtype=float)
    X = X.to_numpy(dtype=float)
    
    gs = (X * W).sum(axis=1) / (W.sum() + 1e-12) ## avoid dividing by zero
    sorted_gs = np.argsort(gs)[::-1][:top_k]          ## returns the top 5 rows of the most similar cases
    # best_idx = np.argmax(gs)
    scores = gs[sorted_gs]
    
    return scores, sorted_gs, gs


# def linear_fxn(Z, activation):
    
#     if activation == 'softmax':
#         ez = np.exp(Z)
#         sm = ez / np.sum(ez)
    
#     elif activation == 'sigmoid':

#         sm = 1 / (1 + np.exp(-Z))
    
#     return sm

def linear_fxn(Z, activation):
    Z = np.asarray(Z, dtype=float)

    if activation == 'softmax':
        Z = Z - np.max(Z)
        ez = np.exp(Z)
        return ez / (np.sum(ez) + 1e-12)

    elif activation == 'sigmoid':
        return 1 / (1 + np.exp(-Z))

    else:
        raise ValueError("activation must be 'softmax' or 'sigmoid'")


# retrieve the similar codes with the actual
def retrieval(idx, casebase, y_train): 
    retrieved_x = casebase.iloc[idx].copy()
    retrieved_y = y_train.iloc[idx]
    # retrieved_x['CKD Progression'] = retrieved_y.to_numpy()
    
    return retrieved_x, retrieved_y

## make prediction of the result
# hard retrieval -----
"""
Majority
"""
def reuser(retrieved_y):
    yhat = np.asarray(retrieved_y).astype(int)
    
    return int(np.bincount(yhat).argmax())

#soft retrieval
def reuse(retrieved_y, scores):
    # y = retrieved_y.to_numpy()
    y = np.asarray(retrieved_y)
    
    s = np.asarray(scores)
    
    w = linear_fxn(s, activation='softmax')
    
    p = np.dot(w, y) 
    
    return p


## predict for a sinle case
def prediction(new_case, casebase, y_train,  num_cols, cat_cols, W, top_k = 5):
    ls = local_sims(new_case, casebase, num_cols, cat_cols)
    scores, idx, _ = global_sims(ls, W, top_k)
    _, y_retrieved = retrieval(idx, casebase, y_train)
    # yhat = y_retrieved.value_counts().idxmax()
    # yhat = reuse(y_retrieved)
    yhat = reuse(y_retrieved, scores)
        
    return yhat, idx, scores


#predict for multiple cases
def multi_prediction(new_case, casebase, y_train,  num_cols, cat_cols, W, top_k = 5):
    preds = []
    for _, case in new_case.iterrows():
        yhat, _, _ = prediction(case, casebase, y_train,  num_cols, cat_cols, W, top_k)
        preds.append(yhat)
        
    return np.asarray(preds)


## Cost function

# def log_loss(preds, y):
#     cost = 0.0
#     y = np.asarray(y)
#     m =  y.shape[0]
#     for i in range(m):    
#         cost +=  - y[i] * np.log(preds[i]) - (1 - y[i]) * np.log(1 - preds[i])
#     cost = cost / m
    
#     return cost


def log_loss(preds, y, eps=1e-12):
    preds = np.asarray(preds)
    y = np.asarray(y)
    
    preds = np.clip(preds, eps, 1-eps)
    
    return -np.mean(y * np.log(preds) + (1 - y) * np.log(1 - preds))

    
    
## compute loss
def compute_loss_cbr(X_val, y_val, W, X_train_norm, y_train, num_cols, cat_cols, top_k =5):
    probs = multi_prediction(X_val, X_train_norm, y_train,  num_cols, cat_cols, W, top_k)
    loss_cbr = log_loss(probs, y_val)
    return loss_cbr




### Evaluation 
def evaluation(pred, actual):
        pred = np.asarray(pred)
        actual = np.asarray(actual)
        # pred_class = (pred >= 0.5).astype(int)
        misclassified = (pred != actual).sum()
        fraction_error = misclassified / len(actual)
        
        return misclassified, fraction_error

In [38]:
## called the trained weights
W1 = np.load('model_weights.npy')
W2 = np.load('2model_weights.npy')

In [45]:
preds = []
for _, case in X_test_norm.iterrows():
    ls = local_sims(case, X_train_norm, num_cols, cat_cols)
    scores, idx, gs = global_sims(ls, W2, top_k=3)
    _, retrieved_y = retrieval(idx, X_train_norm, y_train)
    p = reuser(retrieved_y)
    preds.append(p)

In [40]:
X_test.shape

(291, 22)

In [41]:
evaluation(preds, y_test)

(45, 0.15463917525773196)

In [43]:
evaluation(preds, y_test)

(42, 0.14432989690721648)

In [46]:
evaluation(preds, y_test)

(41, 0.140893470790378)

### GRADIENT DESCENT

In [14]:
def finite_diff_grad(loss_fn, W, eps=1e-4):
    W = np.asarray(W)
    
    grad = np.zeros_like(W)    
    
    for j in range(len(W)):
        Wp = W.copy()
        Wm = W.copy()
        Wp[j] += eps
        Wm[j] -= eps
        
        grad[j] = (loss_fn(Wp) - loss_fn(Wm)) / (2 * eps)
        
    return grad
        

In [15]:
def gradient_descent(X_val, y_val, W, alpha, num_iters, X_train_norm, y_train, num_cols, cat_cols, top_k=5, eps=1e-4):
    
    J_history = []
    
    w = copy.deepcopy(W)
    def loss_fn(W_try):
        return compute_loss_cbr(X_val,y_val, W_try, X_train_norm,y_train, num_cols, cat_cols, top_k=top_k)
    
    for i in range(num_iters):
        ## calculate 
        cost = loss_fn(w)
        
        grad = finite_diff_grad(loss_fn, w, eps=eps)
        
        w = w - alpha * grad
        
        
        # if i < 100000:
        #     J_history.append(compute_gradient_cbr(X, y_val, w, X_train_norm, y_train, num_cols, cat_cols, top_k=5))
        
        J_history.append(cost)
            
        if i% math.ceil(num_iters / 100) == 0:
            print(f"Iteration {i:4d}: Cost {J_history[-1]}      ")
            
    return w, J_history

In [59]:
alpha = 0.01
iters = 200

w_out, histo = gradient_descent(X_val_norm, y_val, nweights, alpha, iters, X_train_norm, y_train, num_cols, cat_cols, top_k=5)

Iteration    0: Cost 0.2579141458565357      
Iteration    2: Cost 0.25791404816704655      
Iteration    4: Cost 0.25791395047849297      
Iteration    6: Cost 0.2579138527908748      
Iteration    8: Cost 0.2579137551041922      
Iteration   10: Cost 0.25791365741844485      
Iteration   12: Cost 0.25791355973363295      
Iteration   14: Cost 0.2579134620497563      
Iteration   16: Cost 0.2579133643668149      
Iteration   18: Cost 0.25791326668480863      
Iteration   20: Cost 0.25791316900373756      
Iteration   22: Cost 0.2579130713236016      
Iteration   24: Cost 0.25791297364440063      
Iteration   26: Cost 0.25791287596613477      
Iteration   28: Cost 0.2579127782888037      
Iteration   30: Cost 0.2579126806124077      
Iteration   32: Cost 0.25791258293694647      
Iteration   34: Cost 0.25791248526242005      
Iteration   36: Cost 0.2579123875888284      
Iteration   38: Cost 0.25791228991617143      
Iteration   40: Cost 0.2579121922444491      
Iteration   42: Cost 0.

In [60]:
import numpy as np

# Example weight parameter (replace with your actual weight array)
weights = np.array(w_out)

# Save the weights to a file named 'model_weights.npy'
np.save('2model_weights.npy', weights) 

print("Weights saved to model_weights.npy")

Weights saved to model_weights.npy


In [61]:
w_out

array([0.96151141, 1.00244137, 1.00123219, 1.00157237, 1.05102703,
       1.00234004, 1.00277769, 1.40610332, 0.99782801, 0.99544698,
       0.99658982, 1.44311706, 0.99508233, 1.00142946, 0.99924447,
       0.99507403, 0.95500146, 0.77109575, 0.99644874, 0.39405158,
       1.96658946, 0.0803751 ])